[RAG-Evaluation-Dataset-JA](https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA)に沿ってコンテキストとなる日本語データを集め、評価用データもダウンロードしました。


In [ ]:
!wget -P ../data https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA/resolve/main/documents.csv 

In [5]:
import pandas as pd
df = pd.read_csv("../data/documents.csv", encoding="utf-8")
df.head(3)

,domain,title,page,url,file_name,publisher
0,finance,2021（令和3）年度 生命保険に関する全国実態調査,17,https://www.jili.or.jp/files/research/zenkokuj...,i-xvii.pdf,公益財団法人生命保険文化センター
1,finance,2023年 保険モニタリングレポート【概要】,11,https://www.fsa.go.jp/news/r4/hoken/20230630-2...,01.pdf,金融庁
2,finance,我が国における気候関連リスクによる住宅ローン・ポートフォリオへの影響分析,31,https://www.fsa.go.jp/frtc/seika/discussion/20...,DP2023-2.pdf,金融庁


In [ ]:
import re
from pathlib import Path
import requests
import pandas as pd

def sanitize_filename(filename: str) -> str:
    # ファイル名に使用できない文字を除去
    return re.sub(r'[\\/*?:"<>|]', '_', filename)

# CSVファイルの読み込み
df = pd.read_csv("../data/documents.csv", encoding="utf-8")

base_dir = Path("../data/pdf")
base_dir.mkdir(parents=True, exist_ok=True)

# ブラウザの User-Agent を設定
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# timeoutの設定
timeout_duration = 10

# itertuples を利用して各行を処理（iには1からの連番を付与）
for i, row in enumerate(df.itertuples(index=False), start=0): 
    # 各カラムの値を取得（rowは属性でアクセス可能）
    domain = row.domain
    url = row.url
    title = row.title
    file_name = row.file_name
    # print(i, url)
    # タイトルをサニタイズして、番号付きのファイル名を生成
    if domain == "public":
        sanitized_title = sanitize_filename(title)
        file_path = base_dir / f"{i}_{domain}_{sanitized_title}_file_name={file_name}"

        try:
            response = requests.get(url, timeout=timeout_duration, headers=headers)
            response.raise_for_status()
        except Exception as e:
            print(f"ダウンロード失敗: URL = {url}, エラー: {e}")
            continue
            
        # Content-Type が PDF であることのチェックを追加
        content_type = response.headers.get("Content-Type", "").lower()
        if not content_type.startswith("application/pdf"):
            print(f"PDF ではない可能性: URL = {url} 内容タイプ: {content_type}")
            continue

        file_path.write_bytes(response.content)
        print(f"保存完了: {file_path}")

保存完了: ../data/pdf/53_public_神奈川県企業庁における総合評価方式に関する運用ガイドライン〔建設工事編〕（令和6年6月）_file_name=r06_06_guideline_itaku.pdf
保存完了: ../data/pdf/54_public_自治体の行政手続のオンライン化に係る手順書【第2.0版】_file_name=000857916.pdf
保存完了: ../data/pdf/55_public_デフレ完全脱却のための総合経済対策～日本経済の新たなステージにむけて～（令和５年11月2日）施策例（全体版3）_file_name=20231102_sesaku3.pdf
保存完了: ../data/pdf/56_public_保健所の活用の仕方_file_name=000761091.pdf
保存完了: ../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf
保存完了: ../data/pdf/58_public_地方公共団体の基幹業務システムの統一・標準化 標準仕様書と適合確認に関する考え方_file_name=20231027_policies_local_governments_outline_02.pdf
保存完了: ../data/pdf/59_public_ふるさと納税に係る指定制度の運用について_file_name=000956094.pdf
保存完了: ../data/pdf/60_public_インボイス制度の負担軽減措置のよくある質問とその回答（令和５年３⽉ 31 ⽇時点）_file_name=qa_futankeigen.pdf
保存完了: ../data/pdf/61_public_「臨海副都心まちづくりガイドライン－2022改定－」（令和4年1月）_file_name=rinpukuGL_2022.pdf
保存完了: ../data/pdf/62_public_地域脱炭素ロードマップ～地方からはじまる、次の時代への移行戦略～_file_name=①地域脱炭素ロードマップ.pdf
保存完了: ../data/pdf/63_public_2023（令和５）年 国民生活基礎調査の概況_file_name=10.pd

In [2]:
%pip install -qU datasets 
from datasets import load_dataset

ds = load_dataset("allganize/RAG-Evaluation-Dataset-JA")

/Users/kj/open_notebooks/env310/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
df = ds['test'].to_pandas()
df.head(3)

,question,target_answer,target_file_name,target_page_no,domain,type,alli_gpt-4o_answer,alli_gpt-4o_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,cohere_command-r-plus_answer,cohere_command-r-plus_ox,anything_llm_gpt-4o_answer,anything_llm_gpt-4o_ox,anything_llm_gpt-4o-mini_answer,anything_llm_gpt-4o-mini_ox,anything_llm_gpt-4-turbo_answer,anything_llm_gpt-4-turbo_ox,anything_llm_gpt-4_answer,anything_llm_gpt-4_ox
0,火災保険の収益悪化に対し、損害保険各社はどのような収益改善策を講じているか、具体的な例を挙げ...,火災保険の収益悪化には長期契約が赤字状態であることや自然災害増加に伴う再保険市場のハード化が...,01.pdf,4,finance,paragraph,損害保険各社は、火災保険の収益悪化に対していくつかの収益改善策を講じています。具体的な例とし...,O,申し訳ありませんが、具体的な収益改善策やその影響についての情報は、提供された文脈からは明らか...,X,...,損害保険各社は、火災保険の収益悪化に対し、独自に収益改善策を講じている。具体的な例としては、...,O,損害保険各社は火災保険の収益悪化に対して、以下の具体的な収益改善策を講じています：\n\n1...,O,損害保険各社は火災保険の収益悪化に対し、以下のような収益改善策を講じています。\n\n1. ...,O,損害保険会社が火災保険の収益悪化に対して取り組んでいる改善策には、以下のようなものがあります...,O,文脈からは具体的な収益改善策の詳細は明らかになっていませんが、一部の損害保険会社は参考純率改...,X
1,2017年度から2021年度までのどの年が最も自然災害の保険金支払額が高かったか？また、その...,"2017年度から2021年度まで最も保険金支払額が高かったのは2018年度で、10,678億...",01.pdf,5,finance,image,2017年度から2021年度までの中で、最も自然災害の保険金支払額が高かったのは2018年度...,O,2018年度が最も自然災害の保険金支払額が高かった年でした[2]。その理由は、台風21号によ...,O,...,2018年度です。保険金の支払額は1兆円を超えています。\n\nその理由は、住宅ローンを取り...,X,2017年度から2021年度までの中で最も自然災害の保険金支払額が高かった年は、2018年度...,O,2017年度から2021年度までの期間において、最も自然災害の保険金支払額が高かったのは20...,X,申し訳ありませんが、2017年度から2021年度までの各年度における自然災害の保険金支払額に...,X,提供された文脈では、2017年度から2021年度までの各年度における自然災害の保険金支払額の...,X
2,主要生保のソルベンシー・マージン比率の推移を考慮すると、令和5年3月期の比率が令和3年3月期...,令和3年3月期のソルベンシー・マージン比率は1007.4でしたが、令和5年3月期には944....,01.pdf,6,finance,image,主要生保のソルベンシー・マージン比率の推移を考慮すると、令和5年3月期の比率は令和3年3月期...,O,ソルベンシー・マージン比率は、令和3年3月期から令和4年3月期にかけて低下しましたが、令和5...,X,...,主要生保のソルベンシー・マージン比率は、令和3年3月期は62.8%、令和5年3月期は55.9...,X,主要生保のソルベンシー・マージン比率の推移を考慮すると、令和5年3月期の比率は令和3年3月期...,X,令和5年3月期の主要生保のソルベンシー・マージン比率は、令和3年3月期と比べて概ね改善傾向を...,X,提供された情報には、主要生命保険会社のソルベンシー・マージン比率の推移に関する具体的なデータ...,X,文脈からは、主要生保のソルベンシー・マージン比率の推移についての具体的な情報は提供されていま...,X


In [4]:
eval_df = df[df['target_file_name'] == 'kaisetsushiryou_2024.pdf']
eval_df = eval_df[['question', 'target_answer', 'target_page_no']].reset_index(drop=True)
eval_df.head(3)

,question,target_answer,target_page_no
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,13
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,16
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,21
